In [31]:
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data.dataset import Dataset 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms,models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import shutil
from sklearn.metrics import f1_score   

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [6]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

In [7]:
transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [8]:
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
train_dir = "Data/train/"
test_dir = "Data/test/"
val_dir= "Data/val/"
train_flooded_dir = "Data/train/flooded/"
train_non_flooded_dir = "Data/train/non-flooded/"
val_flooded_dir = "Data/val/flooded/"
val_non_flooded_dir = "Data/val/non-flooded/"
test_flooded_dir = "Data/test/flooded/"
test_non_flooded_dir = "Data/test/non-flooded/"

In [10]:
train_dataset = ImageFolder(train_dir, transforms_train)
test_dataset = ImageFolder(test_dir, transforms_test)
val_dataset = ImageFolder(val_dir, transforms_val)
train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=12, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=8)

In [11]:
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
print('Val dataset size:', len(val_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

Train dataset size: 644
Test dataset size: 140
Val dataset size: 138
Class names: ['flooded', 'non-flooded']


In [12]:
model = models.resnet18(pretrained=True)   #load resnet18 model
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)
model = model.to(device) 
criterion = nn.CrossEntropyLoss()  #(set loss function)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
num_epochs = 10   #(set no of epochs)
start_time = time.time() #(for showing time)

for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    targets = []
    results = []
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        results.append(preds)
        targets.append(labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    results =torch.cat(results, dim=0)
    targets = torch.cat(targets, dim=0)

    results = results.to('cpu').numpy().flatten()
    targets = targets.to('cpu').numpy().flatten()

    f1_value =  f1_score(results, targets)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
    print('F1-Score', f1_value)
    """ Val Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        targets = []
        results = []
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            results.append(preds)
            targets.append(labels)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset) * 100.
        results = torch.cat(results, dim=0)
        targets =torch.cat(targets, dim=0)
        results = results.to('cpu').numpy().flatten()
        targets = targets.to('cpu').numpy().flatten()
        f1_value =  f1_score(results, targets)
        print('F1-Score', f1_value)
        print('[Val #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 0 running
[Train #0] Loss: 0.0687 Acc: 97.2050% Time: 59.8535s
F1-Score 0.9719626168224299
F1-Score 0.9855072463768115
[Val #0] Loss: 0.0220 Acc: 100.0000% Time: 69.2340s
Epoch 1 running
[Train #1] Loss: 0.0370 Acc: 98.6025% Time: 128.8917s
F1-Score 0.9860465116279069
F1-Score 1.0
[Val #1] Loss: 0.0174 Acc: 101.4493% Time: 135.9583s
Epoch 2 running
[Train #2] Loss: 0.0582 Acc: 97.9814% Time: 196.1721s
F1-Score 0.9799072642967542
F1-Score 0.9928057553956835
[Val #2] Loss: 0.0252 Acc: 100.7246% Time: 202.9996s
Epoch 3 running
[Train #3] Loss: 0.0354 Acc: 99.0683% Time: 262.6492s
F1-Score 0.9906832298136646
F1-Score 0.9928057553956835
[Val #3] Loss: 0.0212 Acc: 100.7246% Time: 269.4287s
Epoch 4 running
[Train #4] Loss: 0.0530 Acc: 98.1366% Time: 329.5719s
F1-Score 0.9814241486068113
F1-Score 0.9929078014184397
[Val #4] Loss: 0.0348 Acc: 100.7246% Time: 336.3504s
Epoch 5 running
[Train #5] Loss: 0.0692 Acc: 97.6708% Time: 395.5775s
F1-Score 0.9765990639625585
F1-Score 0.9787234042553